In [ ]:
import numpy as np
from brian2 import *
import matplotlib.pyplot as plt
import os
import pickle
import random
random.seed(67)

In [4]:
# Define parameter ranges

D1_values = np.arange(0.5, 1.9, 0.05)
D2_values = np.arange(0.5, 1.9, 0.05)

r1 = 10 * Hz
r2 = 7.5 * Hz
r3 = 5 * Hz

N = 3
tau_E = 5.0
gl = 10.0
el = -75.0
v_th = -55.0
memc = 200.0
n_input = 100
strength_input = 1.0
current = 50
eta = 1E-3

w_exc = 1.0
w_max = 3.0

# Storage for results
results = {}
results_1 = {}

for D1 in D1_values:
    for D2 in D2_values:
        
        start_scope()
        tau_plus = 40.0
        tau_minus = D1 * tau_plus
        Aplus = 25
        Aminus = -Aplus * D2

        eqs_neurons = '''
        dv/dt = (-(gl*nsiemens)*(v - el*mV) - (g_E * v) + current*pA) / (memc*pfarad) : volt (unless refractory)
        dg_E/dt = -g_E / (tau_E*ms) : siemens
        '''
        G = NeuronGroup(N, model=eqs_neurons, threshold='v > v_th*mV',
                    reset='v = el*mV', refractory=10*ms, method='euler')
        
        eqs_stdp = '''
        w : 1
        dtrace_pre_plus/dt = -trace_pre_plus/(tau_plus*ms) : 1 (clock-driven)
        dtrace_post_minus/dt = -trace_post_minus/(tau_minus*ms) : 1 (clock-driven)
        '''
        eq_on_pre = '''
        trace_pre_plus += 1.0
        w = clip(w + eta * Aminus * trace_post_minus, 0, w_max)
        g_E += w_exc * nS
        '''
        eq_on_post = '''
        trace_post_minus += 1.0
        w = clip(w + eta * Aplus * trace_pre_plus, 0, w_max)
        '''
        # Create synaptic connections
        con = Synapses(G, G, model=eqs_stdp, on_pre=eq_on_pre, on_post=eq_on_post)
        con.connect(i=0, j=1)
        con.connect(i=0, j=2)
        con.connect(i=1, j=0)
        con.connect(i=1, j=2)
        con.connect(i=2, j=0)
        con.connect(i=2, j=1)
        con.w = w_exc

        P1 = PoissonInput(G[0:1], 'g_E', n_input, rate=r1, weight=strength_input * nS)
        P2 = PoissonInput(G[1:2], 'g_E', n_input, rate=r2, weight=strength_input * nS)
        P3 = PoissonInput(G[2:], 'g_E', n_input, rate=r3, weight=strength_input * nS)

        # Store the final synaptic weight for this (D1, D2) pair
        w_mon = StateMonitor(con, 'w', record=True, dt=1*ms)
        spike_mon = SpikeMonitor(G)
        trace_pre_mon = StateMonitor(con, 'trace_pre_plus', record=True, dt=1*ms)
        trace_post_mon = StateMonitor(con, 'trace_post_minus', record=True, dt=1*ms)
        
        run(300 * second, report='text')
        
        # Store results for plots
        results[(D1, D2)] = {
            'w_monitor_ws': np.array(w_mon.w),
            'spike_times': spike_mon.t / ms,
            'spike_indices': spike_mon.i,
            'trace_pre_values': np.array(trace_pre_mon.trace_pre_plus),
            'trace_post_values': np.array(trace_post_mon.trace_post_minus),
            'time': w_mon.t / second
        }
        
        # Store results_1 for motif analysis
        w_monitor_ws = np.array(w_mon.w)
        final_weight = w_mon.w[:, -1]
        final_weight_matrix = np.zeros((3,3))
        final_weight_matrix[0, 1] = final_weight[0]
        final_weight_matrix[0, 2] = final_weight[1]
        final_weight_matrix[1, 0] = final_weight[2]
        final_weight_matrix[1, 2] = final_weight[3]
        final_weight_matrix[2, 0] = final_weight[4]
        final_weight_matrix[2, 1] = final_weight[5]
        
        results_1[(D1, D2)] = {
            'final_weight_matrix': final_weight_matrix,
            'D1': D1,
            'D2': D2
        }

print("Simulation completed. Results stored!!!!")

INFO       No numerical integration method specified for group 'synapses', using method 'exact' (took 0.11s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 300. s
76.4304 s (25%) simulated in 10s, estimated 29s remaining.
154.6845 s (51%) simulated in 20s, estimated 19s remaining.
232.4582 s (77%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 38s


INFO       No numerical integration method specified for group 'synapses_1', using method 'exact' (took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 300. s
75.8061 s (25%) simulated in 10s, estimated 30s remaining.
152.9139 s (50%) simulated in 20s, estimated 19s remaining.
230.0581 s (76%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 39s
Starting simulation at t=0. s for a duration of 300. s
75.9685 s (25%) simulated in 10s, estimated 29s remaining.
153.6907 s (51%) simulated in 20s, estimated 19s remaining.
232.424 s (77%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 38s
Starting simulation at t=0. s for a duration of 300. s
79.7696 s (26%) simulated in 10s, estimated 28s remaining.
160.136 s (53%) simulated in 20s, estimated 17s remaining.
240.9317 s (80%) simulated in 30s, estimated 7s remaining.
300. s (100%) simulated in 37s
Starting simulation at t=0. s for a duration of 300. s
78.2037 s (26%) simulated in 10s, estimated 28s remaining.
157.499 s (52%) simulated in 20s, estimated 18s remaining.
236.5078 s (78%) simulated in 30s, es

INFO       No numerical integration method specified for group 'synapses', using method 'exact' (took 0.05s). [brian2.stateupdaters.base.method_choice]


...

Starting simulation at t=0. s for a duration of 300. s
77.952 s (25%) simulated in 10s, estimated 28s remaining.
156.3652 s (52%) simulated in 20s, estimated 18s remaining.
234.1291 s (78%) simulated in 30s, estimated 8s remaining.
300. s (100%) simulated in 38s
Starting simulation at t=0. s for a duration of 300. s
77.0865 s (25%) simulated in 10s, estimated 29s remaining.
154.2229 s (51%) simulated in 20s, estimated 19s remaining.
229.5926 s (76%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 39s
Starting simulation at t=0. s for a duration of 300. s
77.9818 s (25%) simulated in 10s, estimated 28s remaining.
156.2092 s (52%) simulated in 20s, estimated 18s remaining.
234.6111 s (78%) simulated in 30s, estimated 8s remaining.
300. s (100%) simulated in 38s
Starting simulation at t=0. s for a duration of 300. s
77.5356 s (25%) simulated in 10s, estimated 29s remaining.
156.4956 s (52%) simulated in 20s, estimated 18s remaining.
235.7861 s (78%) simulated in 30s, 

INFO       No numerical integration method specified for group 'synapses', using method 'exact' (took 0.06s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 300. s
77.168 s (25%) simulated in 10s, estimated 29s remaining.
154.3491 s (51%) simulated in 20s, estimated 19s remaining.
231.5143 s (77%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 38s


INFO       No numerical integration method specified for group 'synapses_1', using method 'exact' (took 0.00s). [brian2.stateupdaters.base.method_choice]


Starting simulation at t=0. s for a duration of 300. s
77.7681 s (25%) simulated in 10s, estimated 29s remaining.
155.0799 s (51%) simulated in 20s, estimated 19s remaining.
232.0471 s (77%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 38s
Starting simulation at t=0. s for a duration of 300. s
78.1281 s (26%) simulated in 10s, estimated 28s remaining.
156.5455 s (52%) simulated in 20s, estimated 18s remaining.
235.3292 s (78%) simulated in 30s, estimated 8s remaining.
300. s (100%) simulated in 38s
Starting simulation at t=0. s for a duration of 300. s
77.7743 s (25%) simulated in 10s, estimated 29s remaining.
154.9871 s (51%) simulated in 20s, estimated 19s remaining.
233.7418 s (77%) simulated in 30s, estimated 9s remaining.
300. s (100%) simulated in 38s
Starting simulation at t=0. s for a duration of 300. s
76.3183 s (25%) simulated in 10s, estimated 29s remaining.
153.708 s (51%) simulated in 20s, estimated 19s remaining.
231.5878 s (77%) simulated in 30s, 

In [6]:
import scipy.io

# Dictionary to store final data for MATLAB
matlab_data = {}

# Store (D1, D2) values in an array
D1_D2_list = []
final_matrices_list = []

# Iterate over the results dictionary
for i, values in enumerate(results_1.values(), start=1):
    D1 = values['D1']
    D2 = values['D2']
    final_weight_matrix = values['final_weight_matrix']

    # Store matrix using an index-based name
    var_name = f"matrix_{i}"
    matlab_data[var_name] = final_weight_matrix  # Save the final weight matrix

    # Store the (D1, D2) mapping
    D1_D2_list.append([i, D1, D2])

# Convert (D1,D2) mapping to a NumPy array
matlab_data["D1_D2_mapping"] = np.array(D1_D2_list)

# Save to MATLAB file
scipy.io.savemat('results_1_D1_D2_different_small_range.mat', matlab_data)

print("✅ Successfully saved 'results_1_D1_D2.mat' with only the values.")

✅ Successfully saved 'results_1_D1_D2.mat' with only the values.
